In [4]:
 from geopy.distance import vincenty

In [3]:
"""
readDataInMem: Read data from provided file into a list (one record per line)
               Returns list
"""

def readDataInMem():
    cols=[]
    rows=[]
    with open("../data/testData",'r') as fr:
        for line in fr:
            cols=line.split()
            if cols[-1]=='\\N':
                cols[-1]='0'
            cols=list(map(int,cols))
            rows.append(cols)
    return rows

In [4]:
rows=readDataInMem()

In [5]:

"""
buildPerUserFrndLst: Build dict of {userId:[friendList]} from list provided
                     Returns the built dict
"""

def buildPerUserFrndLst(rows):
    frndMap={}
    for r in rows:
        if r[0] in frndMap:
            prevList=frndMap[r[0]]
            newSet=set(prevList+r[1:])
            frndMap[r[0]]=list(newSet)
        else:
            frndMap[r[0]]=r[1:]
            
            
    return frndMap

In [6]:
frndMap=buildPerUserFrndLst(rows)

In [15]:
"""
recommendFrnd: 1. Read data from file containing friends
               2. Build per user friendList 
               3. Find most probable friend for given seed on the 
                  basis of max mutual friends
"""

def getCandidates(seed,frndMap):
    level1Frnds=[]
    level1Frnds=frndMap[seed] # Friend list for the provided seed (format: List of friend Ids)
    probableFrnd=0
    support=0
    probableFrndDict=getProbableFriendDict(level1Frnds,frndMap,seed)   # candidates for suggestions
    return probableFrndDict

 



"""
frndWithMaxMutualCount: Calculate the occurence of each Id found at level2 (i.e. friends of level1 friends)
                        Returns the userId with max count and count
"""

def getProbableFriendDict(level1Frnds,frndMap,seed):
    level2Frnds= set()
    freqCnt={}
    thisFrndFrnds=[]

    #iterate through level1 friends

    for i in level1Frnds:
        if i in frndMap.keys():        
            thisFrndFrnds=frndMap[i]
                                     #get friendlist of level1 friend
            for j in thisFrndFrnds:  #for each friend l2 of level1 friend l1,
                if j != seed:       # if l1 not one of level1 friend add t to set
                                     # and set frequency to 1. 
                    if j not in level1Frnds: 
                        if j in level2Frnds:
                            freqCnt[j]=freqCnt[j]+1
                        else:
                            level2Frnds.add(j)
                            freqCnt[j]=1
    return freqCnt



    
    

In [16]:
userInfoDict={}
with open("../data/userFeatureMatrix.txt", 'r') as fp:
    for line in fp:
        cols=line.split()
        if cols[0] not in userInfoDict:
            userInfoDict[int(cols[0])]=list(map(float,cols[1:]))

In [19]:
print(str(userInfoDict))

{10207648390994439: [26.0, 42.3578, -71.0617]}


In [28]:
seed=10207648390994439
probableFrndDict=getCandidates(seed,frndMap)
print(str(probableFrndDict))

{761333840587264: 1, 10202939550824451: 1, 10101764971888982: 1, 10201746230124038: 2, 754643311235080: 1, 750078275061264: 1, 10152834062440985: 1, 760571680660508: 1, 10202204907962398: 1, 10203914558804741: 1, 10204719111966896: 1, 10202337963916273: 2, 10205492441004550: 1, 406112739565606: 1, 729833527078507: 1, 10153252686761004: 1, 998051830247469: 3, 10205287042997294: 1, 10201274848029232: 1, 606052915: 1, 755127101192757: 1, 364030657100346: 1, 863829413637694: 1, 10100428308394559: 1, 10153390122910795: 1, 10100458531161676: 1, 10104356181055053: 1, 1097694013628495: 1, 10206144613507666: 1, 10153144788635731: 1, 1029253450463316: 1, 10154961553375354: 1, 10152877936762042: 1, 10204259123710565: 1, 827757967278184: 1, 818605684852331: 1, 10152543086647916: 1, 10203362021565549: 2, 969960629682798: 1, 1092627744102514: 1, 4084418404979: 1, 671033876323960: 1, 10204262496856698: 1, 10103809753734779: 1, 10155369201895037: 1, 145453392547970: 1, 10153395273753219: 1, 1685098874

In [23]:
def buildMatrix(seed):
    matrix=[]
    
    for user in probableFrndDict:
        row=[]
        if user in userInfoDict:
            row.append(user) #id
            #age
            row.append(1/(abs(userInfoDict[user][0]-userInfoDict[seed][0])))
            #location
            row.append(1/(abs(vincenty((userInfoDict[user][1],userInfoDict[user][2]),(userInfoDict[seed][1],userInfoDict[seed][2])).miles)))
            matrix.append(row)
    return matrix

In [29]:
print(str(buildMatrix(10207648390994439)))


None
